In [7]:
# Copyright 2024 Forsuone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Lab for Gemini basic operation

## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.2 MB/s eta 0:00:00


## Authentication to access to the GCP & Google drive

In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()


## Set the environment on GCP Project

In [3]:
# Set the environment on GCP Project
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"
MODEL_NAME="gemini-1.5-flash"

## Vertex AI initialization


In [4]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

# Initialize the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION) # https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

system_instructions = [
 "Respond in Korean",
 "Respond in structured ways"
]

# Access to the generative model.
# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

## Function to get the response

In [5]:
def generate(query:str, stream:bool):
    """
    Generate a response from the model.
    query :query to be sent to the model
    Returns:The generated response.
    """

    # Request body : https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#request
    # parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = GenerationConfig(
        temperature=0.9,
        top_p=1.0,
        top_k=32,
        candidate_count=1,
        max_output_tokens=8192,
    )

    contents = [query]

    # Counts tokens
    print(f"Token Count : {model.count_tokens(contents)}")

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }

    responses = model.generate_content(
        contents,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=stream,
    )

    return responses



## Run example

In [ ]:

from time import perf_counter

t1_start = perf_counter()

prompt = """
          You are a Generative AI specialist to help people understand the concept of AI.
          Explain what's the pros and cons of using AI in the following instructions.

          1. Define what's the Generative AI.
          2. What are differences between typical machine learning and Generative AI.
          3. What are the pros and cons of using Generative AI.
          4. Explain use cases of Generative AI in the manufacturing industry.
"""
stream = False

responses = generate(prompt, stream)

if stream:
  for response in responses:
    print(response.text, end="")
else:
  # Print the model response
  print(f'\n Usage metadata:\n{responses.to_dict().get("usage_metadata")}')
  print(f"\n Finish reason:\n{responses.candidates[0].finish_reason}")
  print(f"\n Safety settings:\n{responses.candidates[0].safety_ratings}")

  display(Markdown(responses.text))

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

